In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori

In [2]:
def performance_replace(input_database,high_low,bar):
    database=input_database.copy(deep=True)
    for i in range(0,len(database)):
        if high_low=='high':
            if database.iloc[i,0]>=bar:
                database.iloc[i,0]='Qualified'
            else:
                database.iloc[i,0]='Not_Qualified'
        else:
            if database.iloc[i,0]<=bar:
                database.iloc[i,0]='Qualified'
            else:
                database.iloc[i,0]='Not_Qualified'
    return database

In [3]:
def material_replace(database,start,finish):
    for i in range (start,finish):
        for j in range(0,len(database)):
            if database.iloc[j,i]==1:
                database.iloc[j,i]=list(database.columns)[i]+'_True'
            else:
                database.iloc[j,i]=list(database.columns)[i]+'_False'
    return database

In [4]:
def inspect(output):
    lhs = [tuple(result[2][0][0])[0] for result in output]
    rhs = [tuple(result[2][0][1])[0] for result in output]
    support = [result[1] for result in output]
    confidence = [result[2][0][2] for result in output]
    lift = [result[2][0][3] for result in output]
    return list(zip(lhs, rhs, support, confidence, lift))
def inspect_3(output):
    return_list=[]
    for result in output:
        for i in range(0,3):
            try:
                lhs = tuple(result[2][i][0])
                rhs = tuple(result[2][i][1])
                support = result[1]
                confidence = result[2][i][2]
                lift = result[2][i][3]
                return_list.append(tuple([lhs, rhs, support, confidence,lift]))
            except Exception as e:
                pass
    return return_list
def find_qualified_2(output_DataFrame):
    new_df=pd.DataFrame(columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
    for i in range(0,len(output_DataFrame)):
        if output_DataFrame.iloc[i,0]=='Qualified' or output_DataFrame.iloc[i,1]=='Qualified':
            current_dict={'Left_Hand_Side':output_DataFrame.iloc[i,0],'Right_Hand_Side':output_DataFrame.iloc[i,1],
                          'Support':output_DataFrame.iloc[i,2], 'Confidence':output_DataFrame.iloc[i,3],
                          'Lift':output_DataFrame.iloc[i,4]}
            new_df = pd.concat([new_df, pd.DataFrame.from_records([current_dict])])
    return new_df
def find_qualified_3(output_DataFrame):
    new_df=pd.DataFrame(columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
    for i in range(0,len(output_DataFrame)):
        if (len(output_DataFrame.iloc[i,0])==1 and output_DataFrame.iloc[i,0][0]=='Qualified') or (len(output_DataFrame.iloc[i,1])==1 and output_DataFrame.iloc[i,1][0]=='Qualified'):
            current_dict={'Left_Hand_Side':output_DataFrame.iloc[i,0],'Right_Hand_Side':output_DataFrame.iloc[i,1],
                          'Support':output_DataFrame.iloc[i,2], 'Confidence':output_DataFrame.iloc[i,3],
                          'Lift':output_DataFrame.iloc[i,4]}
            new_df = pd.concat([new_df, pd.DataFrame.from_records([current_dict])])
    return new_df

In [5]:
Data = pd.read_csv('./Acid_HER.csv')
not_needed_labels_list=['Warning_OER', 'DOI', 'Country', 'Facility', 'Journal_Name', 'IF',
       'Indexed_Date', 'Cited_Time', 'Alkaline_OVP','Neutral_OVP','Remark_Sentence', 'Bug_Note', 'Irregular_Abnormal_Value',
       'Irregular_Multiple_Value', 'Irregular_Multiple_Keywords','Warning_OER.1']
database=Data.drop(labels=not_needed_labels_list,axis=1)

In [6]:
database

,Acid_OVP,Element_1,Element_2,Element_3,Element_4,Element_5,Element_6,Element_7,Element_8,Judge_Nanoparticles,...,Judge_Electrospin,Judge_Hydrothermal,Judge_Microwave,Judge_Spray,Judge_Phys_Chem_Deposition,Judge_Light_Deposition,Judge_Electrochemical_Deposition,Judge_Electrochemical_Corrosion,Judge_Etching,Judge_Sol_Gel
0,107.0,Co,Cr,Fe,Mo,Ni,None,None,None,0,...,0,0,0,0,0,0,0,0,1,0
1,78.0,C,Mo,S,None,None,None,None,None,0,...,0,1,0,0,1,0,0,0,0,0
2,70.0,P,Co,Se,None,None,None,None,None,0,...,0,0,0,0,0,0,0,0,0,0
3,99.0,C,Mo,None,None,None,None,None,None,0,...,0,0,0,0,0,0,0,0,0,0
4,156.0,Ni,None,None,None,None,None,None,None,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,142.5,Mo,S,None,None,None,None,None,None,1,...,0,0,0,0,1,0,0,0,0,0
1247,136.0,Co,Fe,Ni,P,None,None,None,None,0,...,0,0,0,0,0,0,0,0,0,0
1248,130.0,C,Co,N,Ni,None,None,None,None,1,...,0,0,0,0,0,0,0,0,0,0
1249,53.5,Co,P,None,None,None,None,None,None,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
def get_the_frequency_set_2and3(database,high_or_low,bar):  
    replaced_databse=performance_replace(database,high_or_low,bar)
    final_databse=material_replace(replaced_databse,9,59)
    # Intializing the list
    transacts = []
    # populating a list of transactions
    for i in range(0, len(final_databse)): 
        transacts.append(list(final_databse.iloc[i]))
    rule = apriori(transactions = transacts, min_support = 0.004, min_confidence = 0.2, min_lift = 1.2, min_length = 2, max_length = 2)
    rule_3items = apriori(transactions = transacts, min_support = 0.004, min_confidence = 0.2, min_lift = 1.2, min_length = 3, max_length = 3)

    output = list(rule) # returns a non-tabular output
    # putting output into a pandas dataframe
    results = output
    output_DataFrame = pd.DataFrame(inspect(results), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
    qualified_DF=find_qualified_2(output_DataFrame)
    qualified_DF['Support'] = qualified_DF['Support'].astype(float)
    qualified_DF['Confidence'] = qualified_DF['Confidence'].astype(float)
    qualified_DF['Lift'] = qualified_DF['Lift'].astype(float)

    output_3items = list(rule_3items) # returns a non-tabular output
    # putting output into a pandas dataframe
    results_3items = output_3items
    output_DataFrame_3 = pd.DataFrame(inspect_3(results_3items), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
    qualified_DF_3=find_qualified_3(output_DataFrame_3)
    qualified_DF_3['Support'] = qualified_DF_3['Support'].astype(float)
    qualified_DF_3['Confidence'] = qualified_DF_3['Confidence'].astype(float)
    qualified_DF_3['Lift'] = qualified_DF_3['Lift'].astype(float)

    return qualified_DF.nlargest(n = 500, columns = 'Lift'),qualified_DF_3.nlargest(n = 500, columns = 'Lift')

In [8]:
two_set_75,three_set_75=get_the_frequency_set_2and3(database,'low',75)
two_set_50,three_set_50=get_the_frequency_set_2and3(database,'low',50)
two_set_25,three_set_25=get_the_frequency_set_2and3(database,'low',25)

In [9]:
two_set_25.to_csv('two_set_25'+'.csv')

In [10]:
two_set_50.to_csv('two_set_50'+'.csv')

In [11]:
two_set_75.to_csv('two_set_75'+'.csv')

In [12]:
three_set_25.to_csv('three_set_25'+'.csv')
three_set_50.to_csv('three_set_50'+'.csv')
three_set_75.to_csv('three_set_75'+'.csv')